## **Imports**

In [1]:
# %%capture cap
## Call pip install on 1st time only
# pip install geneticalgorithm
# pip install import-ipynb
# pip install git+https://github.com/hyperopt/hyperopt-sklearn.git
# pip install platypus-opt


In [2]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,SelectFromModel,RFE,chi2,mutual_info_classif,f_classif
import matplotlib.pyplot as plt
from sklearn import preprocessing,metrics
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import seaborn as sns
from sklearn.metrics import accuracy_score
from geneticalgorithm import geneticalgorithm as ga
from sklearn.metrics import accuracy_score,roc_auc_score,auc,roc_curve
import math

import import_ipynb
# from FuzzyTools import FRBCS

from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing, extra_trees,random_forest
from hyperopt import tpe

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


# **Parameter Settings**

In [3]:
Path_data ='./data_myron/GSE89843_count_matrix_tmm.csv'
Path_label ='./data_myron/GSE89843_patients_characteristics.csv'

n_folds=5
max_Evals=50000
n_final_features_to_select=5

metric_of_KBest = mutual_info_classif   #f_classif 
k_KBest=100  #1000

step_REF=1


####for SelectFromModel and RFE
# estimator=RandomForestClassifier()
# estimator=AdaBoostClassifier()
estimator=LogisticRegression() 
# estimator=ExtraTreesClassifier(n_estimators=50)

clf = LogisticRegression( solver='sag', multi_class='multinomial',penalty='l2')   #penalty='none'
# clf = AdaBoostClassifier()  # n_estimators=100
n_estimators=100
# clf = RandomForestClassifier(n_estimators)

algorithm_param = {'max_num_iteration': 100,'population_size':10,'mutation_probability':0.1,\
'elit_ratio': 0,'crossover_probability': 0.5,'parents_portion': 0.3,'crossover_type':'one_point','max_iteration_without_improv':None}


# **Methods and Classes**

**Result, PrintResult,MakeDF**

In [4]:
class Result:
    Score_train,Score_test,AUC_train,AUC_test,n_selceted_feature,n_Rules,state=0,0,0,0,0,0,''
    def __init__(self, Score_train=0, Score_test=0,AUC_train=0,AUC_test=0,n_selceted_feature=0,n_Rules=0,state=''):
        self.Score_train,self.Score_test,self.AUC_train,self.AUC_test,self.n_selceted_feature,self.n_Rules,self.state=Score_train,Score_test,AUC_train,AUC_test,n_selceted_feature,n_Rules,state

def PrintResults(results, state,DF_results):
  # print('\n'+ state + '....')
    scores = [r.Score_train for r in results if r.state==state]
    if len(scores)==0:
        S1=np.nan
    else:
        S1=sum(scores)/len(scores)
    scores = [r.Score_test for r in results if r.state==state]
    if len(scores)==0: 
        S2=np.nan 
    else:
        S2=sum(scores)/len(scores)
    scores = [r.AUC_train for r in results if r.state==state]
    if len(scores)==0:
        S3=np.nan
    else:
        S3=sum(scores)/len(scores)
    scores = [r.AUC_test for r in results if r.state==state]
    if len(scores)==0:
        S4=np.nan
    else:
        S4=sum(scores)/len(scores)
    NumOfFeatures = [r.n_selceted_feature for r in results if r.state==state]
    if len(NumOfFeatures)==0:
        S5=np.nan
    else:
        S5=sum(NumOfFeatures)/len(NumOfFeatures)       
    NumOfRules = [r.n_selceted_feature for r in results if r.state==state]   
    if len(NumOfRules)==0:
        S6=np.nan
    else:
        S6=sum(NumOfRules)/len(NumOfRules)
        
    new_row = {'State':state, 'Train Acc':round(S1*100,2), 'Test Acc':round(S2*100,2), 'Train AUC':round(S3*100,2), 'Test AUC':round(S4*100,2), '# Featuress':S5, '# Rules':S6}
    DF_results = DF_results.append(new_row, ignore_index=True) 
  # print('\n'+ state + '....')
  # print('Average Score_train: {}'.format(S1) + '  Average Score_test : {}'.format(S2) + '  NumOfFeatures : {}'.format(S3))
    return DF_results

def MakeDF(data_array,old_df,cols):
    # cols = old_df.columns[selector.get_support()]
    new_df = pd.DataFrame(data_array, index=old_df.index, columns=cols)
    return new_df

**Train Method**

In [5]:
%%capture cap --no-stdout
def Trainning(X_train,X_test,y_train,y_test,clf,state):
    clf=clf.fit(X_train,y_train)
    Score_train=clf.score(X_train,y_train)
    predicted_train=clf.predict(X_train)
    AUC_train= roc_auc_score(y_train, predicted_train)
    
    Score_test=clf.score(X_test,y_test)
    predicted_test=clf.predict(X_test)
    AUC_test = roc_auc_score(y_test, predicted_test)

# print("Initial Train ACC = ",Score_train)
#   fpr, tpr, thresholds = metrics.roc_curve(y_test, predicted_test)
#   AUC_test = auc(fpr, tpr)
  # print("Initial Test ACC = ",Score_test)   
  # print('AUC: %.2f' % AUC)
    if state== 'Fuzzy':
        print('Number Of Rules: ', len(clf.RB))
        return  Result(Score_train,Score_test,AUC_train,AUC_test,X_train.shape[1],len(clf.RB),state)
    else: 
        return  Result(Score_train,Score_test,AUC_train,AUC_test,X_train.shape[1],state)
 

# **-----Read Data-----**

In [6]:
def ReadData(Path_data, Path_label):
    #X, y = load_iris(return_X_y = True)
    df=pd.read_csv(Path_data)
    df_y=pd.read_csv(Path_label)
    X=df.iloc[:, 1:-1]
    y=df_y.iloc[:, 2].replace('Non.cancer', 0).replace('NSCLC', 1)
    return X,y

# **1) Normalization**

In [7]:
def Normalization(X_train,X_test):
  ## 1.Scaling features to a range [0,1]
  ## 2.Standardize features by removing the mean (u=0) and scaling to the same variance,z = (x - u) / s
  scaler = MinMaxScaler().fit(X_train)
  #scaler = StandardScaler(with_mean=False).fit(X_train)
  X_train_new=scaler.transform(X_train)
  X_test_new=scaler.transform(X_test)
  selected_cols=X_train.columns
  return MakeDF(X_train_new,X_train,selected_cols),MakeDF(X_test_new,X_test,selected_cols)   #return train_df,test_df

# **2) FS: K Best**

In [8]:
def KBestFS(X_train , X_test, metric_of_KBest, k_KBest):
  selector = SelectKBest(metric_of_KBest, k=k_KBest)  
  X_train_new=selector.fit(X_train, y_train).transform(X_train)
  X_test_new=selector.transform(X_test)
  selected_cols=X_train.columns.values[selector.get_support()]
  return MakeDF(X_train_new,X_train,selected_cols),MakeDF(X_test_new,X_test,selected_cols)   #return train_df,test_df

def KBestFS_Iterative(X_train,X_test,metric_of_KBest):
        clf = LogisticRegression( solver='sag', multi_class='multinomial',penalty='l2')
        k_KBest=X_train.shape[1]
        X_train_old,X_test_old=KBestFS(X_train,X_test,metric_of_KBest,k_KBest)
        result=Trainning(X_train_old,X_test_old,y_train,y_test,clf,'SelectKBest_'+str(k_KBest))
        old_measure=result.Score_train
        
        while True:
#             print('k_KBest '+str(k_KBest))
#             print('old_measure '+str(old_measure))
            k_old = k_KBest
            k_KBest=math.floor(k_KBest/2)
            X_train_new,X_test_new=KBestFS(X_train_old,X_test_old,metric_of_KBest,k_KBest)
            result=Trainning(X_train_new,X_test_new,y_train,y_test,clf,'SelectKBest_'+str(k_KBest))
            new_measure=result.Score_train
            
            if (abs(new_measure-old_measure)>=0.02): 
                break
            else :
                old_measure=new_measure
                X_train_old,X_test_old=X_train_new,X_test_new
            
        k_KBest=k_old
        
        while True:
#             print('k_KBest '+str(k_KBest))
#             print('old_measure '+str(old_measure))
            k_KBest=k_KBest-100
            X_train_new,X_test_new=KBestFS(X_train_old,X_test_old,metric_of_KBest,k_KBest)
            result=Trainning(X_train_new,X_test_new,y_train,y_test,clf,'SelectKBest_'+str(k_KBest))
            new_measure=result.Score_train
            
            if (abs(new_measure-old_measure)>=0.01): 
                break
            else :
                old_measure=new_measure
                X_train_old,X_test_old=X_train_new,X_test_new     
                
                
        return X_train_old,X_test_old 
        

# **3) FS: SelectFromModel**

In [9]:
def ByModelFS(X_train , X_test):
  selector = SelectFromModel(estimator)  
  X_train_new=selector.fit(X_train, y_train).transform(X_train)
  X_test_new=selector.transform(X_test)
  selected_cols=X_train.columns.values[selector.get_support()]
  return MakeDF(X_train_new,X_train,selected_cols),MakeDF(X_test_new,X_test,selected_cols)   #return train_df,test_df

# **4) FS: RFE**

In [10]:
def RFE_FS(X_train , X_test, n_final_features_to_select, step_REF):
  selector = RFE(estimator, n_final_features_to_select, step=step_REF)
  X_train_new=selector.fit(X_train, y_train).transform(X_train)
  X_test_new=selector.transform(X_test)
  selected_cols=X_train.columns.values[selector.get_support()]
  return MakeDF(X_train_new,X_train,selected_cols),MakeDF(X_test_new,X_test,selected_cols)   #return train_df,test_df

# **5) HyperoptEstimator**

In [11]:
%%capture cap
# Under Development......
def Tuning_classifier(clf):
  clf1 = HyperoptEstimator(classifier=clf,trial_timeout=300)
  # print( clf1.best_model() )
  return clf1

# **6) Fuzzy Implementation**

## FRBCS_Mine

In [12]:
class FRBCS:
  bins = []
  ABC=[]
  numOfLabels=3
  RB=[]
  def __init__(self,numOfLabels=3):
    self.numOfLabels=numOfLabels
    self.bins = [-0.1]
    numChaumck=((self.getNumOfLabels()-2)*2+2)
    self.partitionLenght = 1/numChaumck   # assumed all data are in [0,1]
    self.bins.extend([x*self.partitionLenght for x in range(1,numChaumck,2)]+[1.1])
    self.ABC=self.setMFsParams()
    return
  def getNumOfLabels(self):
    return self.numOfLabels
    
  def getMFsBins(self):
    return self.bins

  def setMFsParams(self):    
    ABC=[]
    p=(1/((self.getNumOfLabels()-2)*2+2))*2
    ABC.append([0-p,0,0+p])
    for i in range(self.getNumOfLabels()-1):
        a,b=ABC[i][1],ABC[i][2]
        ABC.append([a,b,b+p])
    return ABC

  def getMFsParams(self):     
    return self.ABC
  
  def findLabel(self,value):
    label=np.digitize(value, self.bins, right=True)
    return label
            
  def transformToFuzzyLabels(self, X_train,y_train):     
    df=X_train.copy()
    for col in df.columns:
      df[str(col)+"_Fu"] = df[col].map(lambda a: self.findLabel(a))
      df = df.drop(col, 1)     
    df=df.drop(df.std()[(df.std() == 0)].index, axis=1)   # remove column with totatally same lable
    df['Antecedants'] = df.values.tolist()   # combine Labels
    df['Label']=y_train.copy()                 
    df['AntecedantsLabel'] = df.apply(lambda x: x['Antecedants']+ [x['Label']],axis = 1) # combine Labels
    return df

  def generateRules(self,df):
        df=df.groupby(df['AntecedantsLabel'].map(tuple),as_index =False).agg(Antecedants=('Antecedants','first'),Label=('Label','first'),SP=('AntecedantsLabel','count'),Rules=('AntecedantsLabel','first'))
        df = df[df.groupby(df['Antecedants'].map(tuple))['SP'].transform(max) == df['SP']]
        return  df

  def fit(self,X_train,y_train):
        X_train_fuzzy=self.transformToFuzzyLabels(X_train,y_train)
        df=X_train_fuzzy.copy()
        self.RB=self.generateRules(df)
        return  self   
    
  def score(self,X,y_actual):
        y_predict=self.predict(X)
#         print(y_predict)
#         y_predict=y_actual
        score=accuracy_score(y_actual, y_predict, normalize=True)
        return score
    
  def trimMf(self,z,mf_ind):
            a,b,c=self.ABC[mf_ind-1][0],self.ABC[mf_ind-1][1],self.ABC[mf_ind-1][2]
            if (z<=a) or (z>=c):
                y = 0
            elif (a <= z <= b):
                y = (z-a) / (b-a)
            elif (b <= z <= c):
                y = (c-z) / (c-b)
            return y

  def ClacMFValue(self,rule_Ant,X_row_val):
    mul=1
    for mf_ind, val in zip(rule_Ant, X_row_val):
        mul=mul*self.trimMf(val,mf_ind)
        if mul==0:
           return 0
    return mul   
    
  def classify(self,X_row_val):
        MFs=self.RB.apply(lambda rule: [self.ClacMFValue(rule['Antecedants'],X_row_val),rule['Label']], axis=1)
        predicted_label= max(MFs, key=lambda c: c[0])
        return predicted_label[1]
    
  def predict(self,X):      
      y=X.apply(lambda x: self.classify(x), axis=1)
      return y


## ChiRWClassifier

In [13]:
import time
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\Implementation\\ChiFRBCSPy")
    
from ChiRWClassifier import ChiRWClassifier
  
def Traing_Chi_Alberto(X_tr,X_tst,y_tr,y_tst,state):

    start_time = time.time()
    chi = ChiRWClassifier(frm="wr")

    chi=chi.fit(X_tr,y_tr)
    y_pred = chi.predict(X_tr)
    Score_train=accuracy_score(y_tr,y_pred)
    print("The accuracy of Chi-FRBCS model (train) is: ",Score_train )
    y_pred = chi.predict(X_tst)
    Score_test=accuracy_score(y_tst,y_pred)
    print("The accuracy of Chi-FRBCS model (test) is: ", Score_test)

    #Only for two-class problems
    probas_ = chi.predict_proba(X_tr)
    fpr, tpr, thresholds = roc_curve(y_tr, probas_[:, 1])
    AUC_train = auc(fpr, tpr)
    print("The AUC of Chi-FRBCS model  (train) es: ", AUC_train)
    
    
    probas_ = chi.predict_proba(X_tst)
    fpr, tpr, thresholds = roc_curve(y_tst, probas_[:, 1])
    AUC_test = auc(fpr, tpr)
    print("The AUC of Chi-FRBCS model  (test) es: ", AUC_test)
    
    NumberofRules=len(chi.kb.ruleBase)

    t_exec = time.time() - start_time
    hours = int(t_exec / 3600);
    rest = t_exec % 3600;
    minutes = int(rest / 60);
    seconds = rest % 60;

    print("Execution Time: ", hours , ":" , minutes , ":" , '{0:.4g}'.format(seconds))
    return  Result(Score_train,Score_test,AUC_train,AUC_test,X_train.shape[1],NumberofRules,state)

# **7) Trying a Genetic FS ...**

In [14]:
# is not good enough yet....
def Trainning_GA(X_train,y_train,clf):
  clf.fit(X_train,y_train)
  Score_train=clf.score(X_train,y_train)
  return  Score_train

def f(X):
  # print(X)
  X_tra=X_train.copy()
  l_initial=X_train.shape[1]

  X_tra=X_tra.loc[:,X==1]
  l=X_tra.shape[1]
  print(l)
  s=Trainning_GA(X_tra,y_train,clf)
  print(s)
  return (l/l_initial)+s

# Trying SKMoefs (P.Ducange 2014)...
https://github.com/GionatanG/skmoefs

In [15]:
#%%capture cap

# import sys
# sys.path.append("C:/Users/MASNA.CO/skmoefs/skmoefs")

from platypus.algorithms import *
from skmoefs.toolbox import MPAES_RCS, load_dataset, normalize
from skmoefs.rcs import RCSInitializer, RCSVariator
from skmoefs.discretization.discretizer_base import fuzzyDiscretization
from sklearn.model_selection import train_test_split

def CreatDatFile(X,y,datastName):
    df=X.copy()
    df['Class']=y.copy()
    f=open('dataset/' + datastName +'.dat', 'w')
    f.write('@relation '+datastName +'\n')
    inputs=[]
    line2=""
    for col in X.columns:
        line='@attribute '+ col +' real ' +'['+ str(X[col].values.min()) +','+ str(X[col].values.max())+']\n'
        line2= line2+ str(col)+', '      
        f.write(line) 
        
    line='@attribute Class'+' integer ' +'{'+ str(y.min()) +','+ str(y.max())+'}\n'

    f.write(line)
    f.write('@inputs '+line2+'\n') 
    f.write('@outputs Class\n@data\n') 
    df.to_csv(f,index=False, header=False)
    return 

def TrainMOEA(X_train,y_train,X_test,y_test,foldNumber,state,max_Evals):
    dataset_Tra_Name='GeneExpre_Tra_'+str(foldNumber)
    CreatDatFile(X_train,y_train,dataset_Tra_Name)
    X_Tra, y_Tra, attributes, inputs, outputs = load_dataset(dataset_Tra_Name)  #iris
    Xtr, ytr = normalize(X_Tra, y_Tra, attributes)

    dataset_Tst_Name='GeneExpre_Tst_'+str(foldNumber)
    CreatDatFile(X_test,y_test,dataset_Tst_Name)
    X_Tst, y_Tst, attributes, inputs, outputs = load_dataset(dataset_Tst_Name)  #iris
    Xte, yte = normalize(X_Tst, y_Tst, attributes)

    my_moefs = MPAES_RCS(variator=RCSVariator(), initializer=RCSInitializer())
    my_moefs.fit(Xtr, ytr, max_evals=max_Evals)

    # my_moefs.show_pareto()
    # my_moefs.show_pareto(Xte, yte)
    # my_moefs.show_model('median', inputs=inputs, outputs=outputs)
    scores_train=my_moefs.score(Xtr, ytr)
    scores_test=my_moefs.score(Xte, yte)

    Score_train = max(scores_train, key=lambda x:x[0])[0]  #0 accuracy  1 auc   2 trl
    Score_test= max(scores_test, key=lambda x:x[0])[0]  #0 accuracy  1 auc   2 trl
    AUC_train = max(Score_train, key=lambda x:x[1])[1]
    AUC_test = max(scores_test, key=lambda x:x[1])[1]
#     print(type(Score_train))
#     print(Score_test)
#     print(AUC)

    return  Result(Score_train,Score_test,AUC_test,AUC_test ,X_train.shape[1],state)

# **-----Running-----**

In [ ]:
%%capture cap --no-stdout
results=[]

X,y = ReadData(Path_data, Path_label)

cv= KFold(n_folds, random_state=42, shuffle=True)  #KFold, StratifiedKFold to be check?
i=1
for train_index, test_index in cv.split(X):

    print('fold '+str(i)+' is processing...')
    i=i+1
    X_train,X_test,y_train,y_test=X.iloc[train_index,:],X.iloc[test_index,:],y.iloc[train_index],y.iloc[test_index]

    result=Trainning(X_train,X_test,y_train,y_test,clf,'Initialization')
    results.append(result)

    X_train,X_test=Normalization(X_train,X_test)
    result=Trainning(X_train,X_test,y_train,y_test,clf,'Normalization')
    results.append(result)

#     X_train,X_test=KBestFS(X_train,X_test,metric_of_KBest,k_KBest)
    X_train,X_test=KBestFS_Iterative(X_train,X_test,metric_of_KBest)
    result=Trainning(X_train,X_test,y_train,y_test,clf,'SelectKBest')
    results.append(result)

    X_train,X_test=ByModelFS(X_train,X_test)
    result=Trainning(X_train,X_test,y_train,y_test,clf,'SelectFromModel')
    results.append(result)

    X_train,X_test=RFE_FS(X_train , X_test, n_final_features_to_select, step_REF)
    result=Trainning(X_train,X_test,y_train,y_test,clf,'RFE_FS')
    results.append(result)

    # model=ga(function=f,dimension=X_train.shape[1],variable_type='bool',algorithm_parameters=algorithm_param)
    # model=ga(function=f,dimension=X_train.shape[1],variable_type='bool')
    # model.run()

#     clf1=Tuning_classifier(clf)
#     result=Trainning(X_train,X_test,y_train,y_test,clf1,'Tuning_classifier')
#     results.append(result)

    frbcs_clf=FRBCS(numOfLabels=3).fit(X_train,y_train)
    result=Trainning(X_train,X_test,y_train,y_test,frbcs_clf,'Fuzzy')
    results.append(result)
    
    result=Traing_Chi_Alberto(X_train,X_test,y_train,y_test,'Fuzzy_alberto')
    results.append(result)
    
#     result= TrainMOEA(X_train,y_train,X_test,y_test,i-1,"MPAES_RCS",max_Evals)
#     results.append(result)



fold 1 is processing...
Number Of Rules:  35
Initializing classifier:

Rule Generation
Computing Matching Degrees Rule
Computing Matching Degrees All
Computing Rule Weights
Rule found: IF  10000 THEN 1 RW: 0.38796513450529585
Rule found: IF  11001 THEN 0 RW: 0.17628032850191147
Rule found: IF  01001 THEN 0 RW: 0.48192034542515483
Rule found: IF  01101 THEN 0 RW: 0.2975283490581382
Rule found: IF  11000 THEN 1 RW: 0.10124935629877235
Rule found: IF  00000 THEN 0 RW: 0.0013727563294688122
Rule found: IF  01000 THEN 0 RW: 0.280754423915537
Rule found: IF  00001 THEN 0 RW: 0.2560569621683724
Rule found: IF  00100 THEN 1 RW: 0.4214552880741242
Rule found: IF  00011 THEN 0 RW: 0.6983841307625894
Rule found: IF  00101 THEN 1 RW: 0.11485270661189682
Rule found: IF  02001 THEN 0 RW: 0.7950011802870268
Rule found: IF  00010 THEN 0 RW: 0.5649112936268993
Rule found: IF  01002 THEN 0 RW: 0.9026022746662498
Rule found: IF  00110 THEN 0 RW: 0.033392206378854206
Rule found: IF  10011 THEN 0 RW: 0.413

In [ ]:
(34+39+28+45+39)/5

# **Show Results**

In [ ]:
Results_df = pd.DataFrame(columns=['State', 'Train Acc', 'Test Acc','Train AUC','Test AUC','# Featuress','# Rules'])
Results_df=PrintResults(results, 'Initialization',Results_df)
Results_df=PrintResults(results, 'Normalization',Results_df)
Results_df=PrintResults(results, 'SelectKBest',Results_df)
Results_df=PrintResults(results, 'SelectFromModel',Results_df)
Results_df=PrintResults(results, 'RFE_FS',Results_df)
Results_df=PrintResults(results, 'Tuning_classifier',Results_df)
Results_df=PrintResults(results, 'Fuzzy',Results_df)
Results_df=PrintResults(results, 'Fuzzy_alberto',Results_df)
Results_df=PrintResults(results, 'MPAES_RCS',Results_df)
# Results_df.to_csv("./drive/MyDrive/Colab Notebooks/Results/RF_Results.csv")
Results_df.head(10)

In [ ]:
## With RF as classifier
# Results_df=pd.read_csv("./drive/MyDrive/Colab Notebooks/Results/RF_Results.csv")
# Results_df=Results_df.drop(columns=['Unnamed: 0']).head(10)

# **Plotting**

In [ ]:
g=sns.catplot(x="State", y="Test Acc",kind="point", data=Results_df,color='r')
g.set_xticklabels(label='',rotation=70,size=15)
g.set(ylim=(70, 90))
g.set_axis_labels("","")
g.fig.suptitle('Trend of Test ACC', size=15,color='b')
plt.show()

# ------------End of Pipeline----